In [3]:
import numpy as np
import matplotlib.pyplot as plt
# import util-fns

plt.rcParams['text.usetex'] =  True

# Homework #2
Mangan, Shmoolik, and Uri Alon. "Structure and function of the feed-forward loop network motif." Proceedings of the National Academy of Sciences 100.21 (2003): 11980-11985.

In [ ]:
def f(u, K, H, reg):
    if reg == act or reg == 1:
        return ((u/K)**H)/(1+(u/K)**H)
    if reg == rep or reg == 0:
        return 1/(1+(u/K)**H)

def AND(x1, Kxz, y1, Kyz, H, regX, regY):
    return f(x1, Kxz, H, regX)*f(y1, Kyz, H, regY)
    
def OR(x1, Kxz, y1, Kyz, H, regX, regY):
    def fc(u, Ku, v, Kv, reg):
        if reg==1 or reg==act:
            return (u/Ku)**H/(1+ (u/Ku)**H + (v/Kv)**H)
        if reg==0 or reg==rep:
            return 1/(1+ (u/Ku)**H + (v/Kv)**H)
    gate = fc(x1, Kxz, y1, Kyz, H, regX, regY) + fc(y1, Kyz, x1, Kxz, H, regX, regY)
    return gate